In [1]:
import numpy as np
import pandas as pd, re, requests
from itertools import combinations
from scipy.sparse import dok_matrix
import matplotlib.pyplot as plt
import seaborn as sns
import os
import re
from tqdm.auto import tqdm
import seaborn as sns, matplotlib.pyplot as plt
from sklearn.metrics import jaccard_score
from sklearn.preprocessing import MultiLabelBinarizer

/Users/sofsen/miniconda3/envs/my_env/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
co_df = pd.read_csv("matrix/cat_co_matrix_counts.csv") # сколько уникальных пациентов имели одновременно диагнозы i и j
fraction_df = pd.read_csv("matrix/cat_co_matrix_fraction.csv") # доля пациентов из всей выборки у которых одновременно были диагнозы i и j
cond_df= pd.read_csv("matrix/cat_co_matrix_conditional_prob.csv") # усл вероятность

df = pd.read_csv('diagnoses_icd10.csv')

In [ ]:
# Jaccard Similarity -- посчитать насколько похожи диагнозы по набору пациентов, у которых они встречаются

# df_unique = df[['subject_id', 'icd10_category']].drop_duplicates()
# pivot = df_unique.groupby('subject_id')['icd10_category'].apply(set)

# mlb = MultiLabelBinarizer()
# binary_matrix = mlb.fit_transform(pivot)

# from sklearn.metrics import pairwise_distances
# jaccard_matrix = 1 - pairwise_distances(binary_matrix.T, metric='jaccard')

# jaccard_df = pd.DataFrame(jaccard_matrix, index=mlb.classes_, columns=mlb.classes_)
# jaccard_df.head()

In [5]:
df_unique = df[['subject_id', 'icd10_category']].drop_duplicates()
pivot = df_unique.groupby('subject_id')['icd10_category'].apply(set)

In [7]:
df['icd10_category'].nunique()

1757

In [ ]:
mlb = MultiLabelBinarizer()
binary_matrix = mlb.fit_transform(pivot)
binary_dense = binary_matrix.astype(bool).T  # T: диагнозы как строки

n = binary_dense.shape[0]
jaccard_result = np.zeros((n, n))

for i in tqdm(range(n)):
    for j in range(i, n):
        score = jaccard_score(binary_dense[i], binary_dense[j])
        jaccard_result[i, j] = score
        jaccard_result[j, i] = score  # симметрично

# Формируем DataFrame
jaccard_df = pd.DataFrame(jaccard_result, index=mlb.classes_, columns=mlb.classes_)


 21%|██        | 361/1757 [8:20:04<31:17:12, 80.68s/it]   

In [ ]:
jaccard_df.to_csv('stat/jaccard_g.csv')

In [ ]:
plt.figure(figsize=(15,12))
sns.heatmap(jaccard_df, annot=False, cmap="viridis")
# plt.title(name, fontsize=20)
# plt.xticks(rotation=90, fontsize=8)
# plt.yticks(rotation=0, fontsize=8)
plt.savefig(f'pics/jaccard.jpg', dpi=300, bbox_inches='tight')
plt.tight_layout()
plt.show()

In [ ]:
# PMI какие коды сильно информативны друг о друге (например, редкие, но часто встречающиеся вместе) - Pointwise Mutual Information

total_patients = df['subject_id'].nunique()
co_occurrence = cond_df * co_df.values.diagonal()[:, None]  # восстанавливаем N_{i,j}

P_i = co_df.values.diagonal() / total_patients
P_j = co_df.values.diagonal() / total_patients
P_ij = co_occurrence / total_patients

with np.errstate(divide='ignore'):
    pmi = np.log2(P_ij / (P_i[:, None] * P_j[None, :]))

pmi_df = pd.DataFrame(pmi, index=cond_df.index, columns=cond_df.columns)
pmi_df.to_csv('stat/pmi_df_g.csv')


In [ ]:
# какие болезни предшествуют другим
df_sorted = df.sort_values(['subject_id', 'admittime'])

df_sorted['next_code'] = df_sorted.groupby('subject_id')['icd10_category'].shift(-1)
df_sorted['next_time'] = df_sorted.groupby('subject_id')['admittime'].shift(-1)

#  сколько раз диагноз i предшествовал диагнозу j
df_transitions = df_sorted.dropna(subset=['icd10_category', 'next_code'])
df_transitions = df_transitions[df_transitions['icd10_category'] != df_transitions['next_code']]

transition_counts = df_transitions.groupby(['icd10_category', 'next_code']).size().reset_index(name='count')
transition_counts.sort_values('count', ascending=False).head(10)
transition_counts.to_csv('stat/transition_counts.csv')